In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [2]:
pref = 'salud'

In [3]:
df1 = pd.read_csv('reto_precios.csv')
df2 = pd.read_csv('saluds.csv', encoding='latin-1')

C:\Users\alber\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df2.head()

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,9331895,01001621111005611000000000U0,"(H) CONSULTORIO AGUASCALIENTES, AGS.","(H) CONSULTORIO AGUASCALIENTES, AGS.",621111,Consultorios de medicina general del sector pr...,0 a 5 personas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0731,3,NaN,NaN,NaN,Fijo,21.879700,-102.275355,2020-11
1,6697315,01001621511001384002003549S9,176 SUCURSAL EMILIANO ZAPATA SIGLO XXI,GRUPO DIAGNOSTICO MEDICO PROA SA DE CV,621511,Laboratorios médicos y de diagnóstico del sect...,31 a 50 personas,CALLE,GENERAL EMILIANO ZAPATA,CALLE,JOSÉ GUADALUPE POSADA,CALLE,AZTECA,CALLE,TALAMANTES,532.0,NaN,NaN,NaN,0.0,NaN,COLONIA,SAN MARCOS,NaN,NaN,NaN,20070.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0619,9,NaN,ERICK_ROQUE@PROA.COM.MX,WWW.CHOPO.COM.MX,Fijo,21.882548,-102.306592,2016-01
2,6751037,01001621511001441000003549S7,184 SUCURSAL CONVENCIÓN ORIENTE GRUPO DIAGNÓ...,GRUPO DIAGNOSTICO MEDICO PROA SA DE CV,621511,Laboratorios médicos y de diagnóstico del sect...,0 a 5 personas,AVENIDA,CONVENCIÓN DE 1914 ORIENTE,AVENIDA,HALEY,AVENIDA,PASEO DE LA CRUZ,CALLE,VÍA LÁCTEA,702.0,NaN,NaN,NaN,0.0,NaN,COLONIA,JESUS GOMEZ PORTUGAL,OTRO CONGLOMERADO,LA LOMA,702-A,20250.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,2189,5,NaN,ERICK_ROQUE@PROA.COM.MX,NaN,Fijo,21.873174,-102.270598,2016-10
3,536,01003624191000011001000000S3,AA GRUPO CHICAGO 85,NaN,624191,Agrupaciones de autoayuda para alcohólicos y p...,11 a 30 personas,CALLE,ABASOLO,CALLE,INDEPENDENCIA,CALLE,5 DE MAYO,CALLE,FRANCISCO JAVIER MINA,NaN,SN,NaN,NaN,NaN,NaN,COLONIA,INDEPENDENCIA,NaN,NaN,NaN,20800.0,1,Aguascalientes,3,Calvillo,1,Calvillo,0187,24,4951077786,NaN,NaN,Fijo,21.845172,-102.722171,2010-07
4,8625042,01001624191002591000000000U6,AA GRUPO LA SOLEDAD,CENTRAL MEXICANA DE SERVICIOS GENERALES AA AC,624191,Agrupaciones de autoayuda para alcohólicos y p...,0 a 5 personas,CALLE,ATOTONILCO,CALLE,TENACATITA,CALLE,SANTOS DEGOLLADO,CALLE,AJIJIC,201.0,NaN,NaN,NaN,NaN,NaN,FRACCIONAMIENTO,LA SOLEDAD,NaN,NaN,NaN,20326.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,2846,15,NaN,NaN,NaN,Fijo,21.877198,-102.333976,2019-11


In [5]:
df2.columns

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [6]:
df1 = df1[['id','lat','lon']]
# Cambiar los nombres de las variables en df1 agregando sufijo 1
df1 = df1.rename(columns={ 'lat': 'lat1', 'lon': 'lon1'})
# Cambiar los nombres de las variables en df2 agregando sufijo 2
df2 = df2.rename(columns={'id': 'id2', 'latitud': 'lat2', 'longitud': 'lon2'})


In [7]:
# Definir la tolerancia espacial (en kilómetros)
tolerancia = 0.5

# Convertir las latitudes y longitudes a radianes
df1['lat_rad1'] = df1['lat1'].apply(radians)
df1['lon_rad1'] = df1['lon1'].apply(radians)
df2['lat_rad2'] = df2['lat2'].apply(radians)
df2['lon_rad2'] = df2['lon2'].apply(radians)

In [8]:
# Calcular la distancia haversine entre las coordenadas
distances = haversine_distances(df1[['lat_rad1', 'lon_rad1']], df2[['lat_rad2', 'lon_rad2']]) * 6371000/1000  # Convertir a kilómetros

# Encontrar las coincidencias dentro de la tolerancia
matches = distances <= tolerancia

In [9]:
matching_indices = np.where(matches)

# Crear una lista para almacenar los DataFrames combinados
combined_dfs = []

# Iterar sobre los índices y columnas de coincidencias
for idx, col in zip(*matching_indices):
    # Obtener los registros coincidentes de df1 y df2
    matching_record_df1 = df1.loc[idx]
    matching_record_df2 = df2.loc[col]
    
    # Combinar los registros en un DataFrame
    combined_record = pd.concat([matching_record_df1, matching_record_df2])
    combined_dfs.append(combined_record)

# Concatenar todos los DataFrames combinados en uno solo
final_combined_df = pd.concat(combined_dfs, axis=1).T

In [10]:
final_combined_df.head(3)

,id,lat1,lon1,lat_rad1,lon_rad1,id2,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,lat2,lon2,fecha_alta,lat_rad2,lon_rad2
0,1.0,19.421317,-99.164182,0.338966,-1.730741,6712346,09015621511002063001003549S0,171 LABORATORIO MEDICO DEL CHOPO,GRUPO DIAGNOSTICO MEDICO PROA SA DE CV,621511,Laboratorios médicos y de diagnóstico del sect...,0 a 5 personas,EJE VIAL,2 PONIENTE MONTERREY,CALLE,TABASCO,AVENIDA,ÁLVARO OBREGÓN,CALLE,TONALÁ,104.0,NaN,NaN,NaN,0.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6760.0,9,Ciudad de México,15,Cuauhtémoc,1,Cuauhtémoc,1002,21,NaN,JUANBRAVO@PROA.COM.MX,WWW.CHOPO.COM.MX,Fijo,19.417652,-99.164139,2016-01,0.338902,-1.730741
1,1.0,19.421317,-99.164182,0.338966,-1.730741,1029531,09015624191000692001000000U6,AGRUPACIONES DE AUTOAYUDA PARA ALCOHOLICOS Y P...,GRUPO PESCADOR ALCOHOLICOS ANONIMOS,624191,Agrupaciones de autoayuda para alcohólicos y p...,11 a 30 personas,CALLE,COLIMA,AVENIDA,INSURGENTES SUR,CALLE,TONALA,CALLE,DURANGO,249.0,NaN,EDIFICIO SIN NOMBRE,PISO 2,0.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,Ciudad de México,15,Cuauhtémoc,1,Cuauhtémoc,1002,16,NaN,NaN,NaN,Fijo,19.419067,-99.16374,2010-07,0.338927,-1.730734
2,1.0,19.421317,-99.164182,0.338966,-1.730741,1027944,09015621113016441000000000U7,ANESTECIA CIBELES,NaN,621113,Consultorios de medicina especializada del sec...,0 a 5 personas,EJE VIAL,3 PONIENTE (MEDELLIN),AVENIDA,PLAZA VILLA MADRID,CALLE,SINALOA,CALLE,SINALOA,43.0,NaN,EDIFICIO SIN NOMBRE,PLANTA BAJA,NaN,A1,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,Ciudad de México,15,Cuauhtémoc,1,Cuauhtémoc,0998,4,NaN,NaN,NaN,Fijo,19.420608,-99.166797,2010-07,0.338954,-1.730787


In [11]:
len(final_combined_df)

372740

In [12]:
final_combined_df['numero_ext'].value_counts()

59.0      92800
29.0      32328
144.0     21397
113.0     13479
16.0      12161
          ...  
1451.0        1
676.0         1
197.0         1
1808.0        1
560.0         1
Name: numero_ext, Length: 444, dtype: int64

In [13]:
final_combined_df['manzana'].value_counts()

17    125184
7      39351
8      38631
13     20375
16     20024
2      17351
14     13034
24     11016
18     10601
15     10139
6       9639
4       7036
1       6448
25      5614
20      5374
12      5216
5       4185
22      3795
19      3761
10      3359
9       2797
11      2517
23      2191
3       2036
21      1204
26      1161
29       325
27       110
28       102
37        42
46        38
31        13
42        12
33        11
41        11
32         7
39         6
75         6
43         6
44         5
30         3
35         2
38         1
34         1
Name: manzana, dtype: int64

In [14]:
final_combined_df['per_ocu'].value_counts()

0 a 5 personas        331093
6 a 10 personas        20568
11 a 30 personas       14320
251 y más personas      2424
51 a 100 personas       2020
31 a 50 personas        1246
101 a 250 personas      1069
Name: per_ocu, dtype: int64

In [15]:
personal_ocupado = {'0 a 5 personas' : 1,
'6 a 10 personas' : 2,
'11 a 30 personas' : 3,
'31 a 50 personas' : 4,
'51 a 100 personas' : 5,
'101 a 250 personas' : 6,
'251 y más personas' : 7}

final_combined_df['per_ocu'] = final_combined_df['per_ocu'].replace(personal_ocupado)
final_combined_df['per_ocu'].value_counts()

1    331093
2     20568
3     14320
7      2424
5      2020
4      1246
6      1069
Name: per_ocu, dtype: int64

In [16]:
#per_ocu
#numero_ext
#manzana

In [17]:
final_combined_df['dif_lat_rad'] =final_combined_df['lat_rad1']-final_combined_df['lat_rad2']
final_combined_df['dif_lon_rad'] =final_combined_df['lon_rad1']-final_combined_df['lon_rad2']

In [18]:
(final_combined_df['lat_rad1']-final_combined_df['lat_rad2'])

0         0.000064
1         0.000039
2         0.000012
3         0.000062
4         0.000058
            ...   
372735   -0.000063
372736   -0.000063
372737   -0.000038
372738    -0.00002
372739   -0.000063
Length: 372740, dtype: object

In [19]:
final_combined_df.entidad.value_counts()

Ciudad de México    371877
Nuevo León             856
Baja California          7
Name: entidad, dtype: int64

In [20]:
final_combined_df[final_combined_df.entidad.str.contains('NUEVO LEÓN')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad


In [21]:
final_combined_df[final_combined_df.entidad.str.contains('BAJA CALIFORNIA')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad


In [22]:
summary = final_combined_df[['id','per_ocu','numero_ext','manzana']]
summary.head()

,id,per_ocu,numero_ext,manzana
0,1.0,1,104.0,21
1,1.0,3,249.0,16
2,1.0,1,43.0,4
3,1.0,1,262.0,11
4,1.0,2,121.0,20


In [23]:
# Definir las columnas para la agrupación y agregación
aggregation_columns = ['per_ocu', 'numero_ext', 'manzana']  # Agrega aquí las columnas que quieras sumar

# Realizar la agrupación y las agregaciones
grouped_df = summary.groupby('id')[aggregation_columns].agg({'sum'}).reset_index()
grouped_df2 = summary.groupby('id').agg(total=('id', 'count')).reset_index()
grouped_df2.columns = ['id'] + [f'{pref}_total']



# Renombrar las columnas resultantes
grouped_df.columns = ['id'] + [f'{pref}_suma_{col}' for col in aggregation_columns]

# Mostrar el DataFrame resultante
centros = grouped_df2.merge(grouped_df)

In [24]:
centros.head()

,id,salud_total,salud_suma_per_ocu,salud_suma_numero_ext,salud_suma_manzana
0,1.0,69,111,158745.0,742
1,2.0,73,134,157613.0,876
2,3.0,214,282,25955.0,3000
3,4.0,102,131,13856.0,1292
4,5.0,311,414,22360.0,1959


In [25]:
centros.to_csv('centros_'+pref+'s'+'.csv')